In [1]:
import pandas as pd
import yaml
import plotly.express as px

### 1. Composers

In [3]:
file=open('../../Lieder/data/composers.yaml','r')
composers_yaml=yaml.safe_load(file)

composers_df=pd.DataFrame(composers_yaml).reset_index().T

new_header = composers_df.iloc[0] #grab the first row for the header
composers_df = composers_df[1:]
composers_df.columns=new_header

composers_df=composers_df.reset_index().drop(columns='index')

composers_df=composers_df.rename(columns={'path':'composer_path',
                             'name':'composer_name',
                             'gender':'composer_gender'})

In [3]:
#composers_df.to_csv('composers.csv',index=False)

In [4]:
import plotly.express as px

fig=px.bar(composers_df.sort_values(by='scores',ascending=False),x='composer_name',y='scores')
#fig.show()

### 2. Scores

In [7]:
file=open('../../Lieder/data/scores.yaml','r')
scores_yaml=yaml.safe_load(file)

scores_df=pd.DataFrame(scores_yaml).reset_index().T

new_header = scores_df.iloc[0] #grab the first row for the header
scores_df = scores_df[1:]
scores_df.columns=new_header

scores_df=scores_df.reset_index().drop(columns='index')

scores_df['composer_path']=scores_df['path'].apply(lambda x: x.split('/')[0])

## merge composers data

scores_df_variables=['path', 'name', 'set_id', 'composer_path']
composers_df_variables=['composer_path', 'composer_name', 'composer_gender', 'desc', 'sets', 'scores']

scores_df=scores_df[scores_df_variables].merge(composers_df[composers_df_variables],on='composer_path')

files_to_delete=['Schubert,_Franz/Die_schöne_Müllerin,_D.795/05_Am_Feierabend','Schubert,_Franz/Die_schöne_Müllerin,_D.795/03_Halt!']

scores_df=scores_df[~scores_df['path'].isin(files_to_delete)].reset_index(drop=True)

## get list of paths of midi files
import json

file=open('../data preprocessing/corpus_conversion_midi.json')
scores_mapping=json.load(file)

scores_paths=[]
tracks_per_score_list_clean=[]
tracks_per_score_list_raw=[]


track_names_mapping_df=pd.read_csv('../dataframes/track_names_mapping_df.csv')

# Get mapping of clean track names
track_names_mapping = {}
for key, value in zip(track_names_mapping_df['track_name_raw'], track_names_mapping_df['track_name_clean']):
    track_names_mapping[key] = value
  
from mido import MidiFile

for score in scores_mapping:    
    
    midi_file='../'+score['out']
    scores_paths.append(midi_file)
    track_names_clean=[]
    track_names_raw=[]

    for track in MidiFile(midi_file).tracks:
        track_names_raw.append(track.name)
        track_names_clean.append(track_names_mapping[track.name])

    tracks_per_score_list_clean.append(track_names_clean)
    tracks_per_score_list_raw.append(track_names_raw)

## add full scores paths as variable
scores_df['scores_paths']=scores_paths

scores_df['track_names_clean']=tracks_per_score_list_clean
scores_df['track_names_raw']=tracks_per_score_list_raw

#scores_df['contains_piano_track?']=scores_df['track_names'].apply(lambda x: 'piano' in x)

KeyError: 'track_names'

In [9]:
scores_df['contains_other_tracks?']=scores_df['track_names_clean'].apply(lambda x: 'other' in x)

In [13]:
songs_w_other_tracks=scores_df[scores_df['contains_other_tracks?']==True]

In [20]:
songs_w_other_tracks=songs_w_other_tracks.explode('track_names_raw')

In [21]:
songs_w_other_tracks

index,path,name,set_id,composer_path,composer_name,composer_gender,desc,sets,scores,scores_paths,track_names_clean,track_names_raw,contains_other_tracks?
68,"Bizet,_Georges/Feuilles_d’album/5_Rose_d’Amour",Rose d’Amour,5079497,"Bizet,_Georges",Georges Bizet,Male,French composer,2,26,"../../Lieder/scores/Bizet,_Georges/Feuilles_d’...","[other, piano, piano]",Oboe,True
68,"Bizet,_Georges/Feuilles_d’album/5_Rose_d’Amour",Rose d’Amour,5079497,"Bizet,_Georges",Georges Bizet,Male,French composer,2,26,"../../Lieder/scores/Bizet,_Georges/Feuilles_d’...","[other, piano, piano]",Grand Piano,True
68,"Bizet,_Georges/Feuilles_d’album/5_Rose_d’Amour",Rose d’Amour,5079497,"Bizet,_Georges",Georges Bizet,Male,French composer,2,26,"../../Lieder/scores/Bizet,_Georges/Feuilles_d’...","[other, piano, piano]",Grand Piano,True
69,"Bizet,_Georges/Feuilles_d’album/6_Le_grillon",Le grillon,5079497,"Bizet,_Georges",Georges Bizet,Male,French composer,2,26,"../../Lieder/scores/Bizet,_Georges/Feuilles_d’...","[voice, other, piano, piano, piano]",Soprano,True
69,"Bizet,_Georges/Feuilles_d’album/6_Le_grillon",Le grillon,5079497,"Bizet,_Georges",Georges Bizet,Male,French composer,2,26,"../../Lieder/scores/Bizet,_Georges/Feuilles_d’...","[voice, other, piano, piano, piano]",Oboe,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,"Zumsteeg,_Emilie/5_Lieder/3_In_das_Stambuch_ei...",In das Stambuch einer Freundin,5103253,"Zumsteeg,_Emilie",Emilie Zumsteeg,Female,"German composer, music teacher, choir conducto...",2,11,"../../Lieder/scores/Zumsteeg,_Emilie/5_Lieder/...","[voice, other]",Guitar,True
1346,"Zumsteeg,_Emilie/5_Lieder/4_Romanze",Romanze,5103253,"Zumsteeg,_Emilie",Emilie Zumsteeg,Female,"German composer, music teacher, choir conducto...",2,11,"../../Lieder/scores/Zumsteeg,_Emilie/5_Lieder/...","[voice, other]",Voice,True
1346,"Zumsteeg,_Emilie/5_Lieder/4_Romanze",Romanze,5103253,"Zumsteeg,_Emilie",Emilie Zumsteeg,Female,"German composer, music teacher, choir conducto...",2,11,"../../Lieder/scores/Zumsteeg,_Emilie/5_Lieder/...","[voice, other]",Guitar,True
1347,"Zumsteeg,_Emilie/5_Lieder/5_Sehnsucht_der_Liebe",Sehnsucht der Liebe,5103253,"Zumsteeg,_Emilie",Emilie Zumsteeg,Female,"German composer, music teacher, choir conducto...",2,11,"../../Lieder/scores/Zumsteeg,_Emilie/5_Lieder/...","[voice, other]",Voice,True


In [27]:
songs_w_other_tracks['track_names_raw'].unique()

array(['Oboe', 'Grand Piano', 'Soprano', 'Piano', 'Alto', 'Viola',
       'Part_1', 'Hautbois', 'Tenor', 'Bass', 'Voice', 'Classical Guitar',
       'Violoncelle', 'Harpsichord', 'Flute', 'Violoncello', 'Violin',
       'Cembalo', 'Soprano II (cutaway stave)', 'Tenor I', 'Tenor II',
       'Bass I', 'Bass II', 'Soprano I', 'Soprano II', 'Soprani II',
       'Guitar'], dtype=object)

In [28]:
tracks_to_exclude=[ 'Grand Piano', 'Soprano', 'Piano', 'Alto', 'Tenor', 'Bass', 'Voice', 'Soprano II (cutaway stave)', 'Tenor I', 'Tenor II',
       'Bass I', 'Bass II', 'Soprano I', 'Soprano II', 'Soprani II']

In [29]:
songs_w_other_tracks[~songs_w_other_tracks['track_names_raw'].isin(tracks_to_exclude)].groupby(by=['track_names_raw','composer_gender']).count().reset_index()

index,track_names_raw,composer_gender,path,name,set_id,composer_path,composer_name,desc,sets,scores,scores_paths,track_names_clean,contains_other_tracks?
0,Cembalo,Female,2,2,2,2,2,2,2,2,2,2,2
1,Classical Guitar,Female,2,2,2,2,2,2,2,2,2,2,2
2,Flute,Female,1,1,1,1,1,1,1,1,1,1,1
3,Guitar,Female,5,5,5,5,5,5,5,5,5,5,5
4,Harpsichord,Female,2,2,2,2,2,2,2,2,2,2,2
5,Hautbois,Male,1,1,1,1,1,1,1,1,1,1,1
6,Oboe,Female,4,4,4,4,4,4,4,4,4,4,4
7,Oboe,Male,4,4,4,4,4,4,4,4,4,4,4
8,Part_1,Female,2,2,2,2,2,2,2,2,2,2,2
9,Part_1,Male,6,6,6,6,6,6,6,6,6,6,6


In [5]:
scores_df

index,path,name,set_id,composer_path,composer_name,composer_gender,desc,sets,scores,scores_paths,track_names,contains_piano_track?
0,"Abbott,_Jane_Bingham/_/Just_for_Today",Just for Today,5106766,"Abbott,_Jane_Bingham",Jane Bingham Abbott,Female,"US pianist, singer and composer",1,2,"../../Lieder/scores/Abbott,_Jane_Bingham/_/Jus...","[voice, piano, piano]",True
1,"Abbott,_Jane_Bingham/_/Think_of_Today",Think of Today,5106766,"Abbott,_Jane_Bingham",Jane Bingham Abbott,Female,"US pianist, singer and composer",1,2,"../../Lieder/scores/Abbott,_Jane_Bingham/_/Thi...","[voice, piano, piano]",True
2,"Abrams,_Harriett/_/Crazy_Jane",Crazy Jane,5106769,"Abrams,_Harriett",Harriett Abrams,Female,"English musician, singer, composer",1,2,"../../Lieder/scores/Abrams,_Harriett/_/Crazy_J...","[voice, piano, piano]",True
3,"Abrams,_Harriett/_/The_Orphan’s_Prayer",The Orphan’s Prayer,5106769,"Abrams,_Harriett",Harriett Abrams,Female,"English musician, singer, composer",1,2,"../../Lieder/scores/Abrams,_Harriett/_/The_Orp...","[voice, piano, piano]",True
4,"Bantock,_Granville/5_Songs_from_the_Chinese_Po...",The Old Fisherman of the Mists and Waters,5103566,"Bantock,_Granville",Granville Bantock,Male,British composer and conductor,1,5,"../../Lieder/scores/Bantock,_Granville/5_Songs...","[voice, piano, piano]",True
...,...,...,...,...,...,...,...,...,...,...,...,...
1349,"Zumsteeg,_Emilie/6_Lieder,_Op.4/2_Morgenfreude",Morgenfreude,5103263,"Zumsteeg,_Emilie",Emilie Zumsteeg,Female,"German composer, music teacher, choir conducto...",2,11,"../../Lieder/scores/Zumsteeg,_Emilie/6_Lieder,...","[voice, piano, piano]",True
1350,"Zumsteeg,_Emilie/6_Lieder,_Op.4/3_Religion",Religion,5103263,"Zumsteeg,_Emilie",Emilie Zumsteeg,Female,"German composer, music teacher, choir conducto...",2,11,"../../Lieder/scores/Zumsteeg,_Emilie/6_Lieder,...","[voice, piano, piano]",True
1351,"Zumsteeg,_Emilie/6_Lieder,_Op.4/4_An_meine_Zither",An meine Zither,5103263,"Zumsteeg,_Emilie",Emilie Zumsteeg,Female,"German composer, music teacher, choir conducto...",2,11,"../../Lieder/scores/Zumsteeg,_Emilie/6_Lieder,...","[voice, piano, piano]",True
1352,"Zumsteeg,_Emilie/6_Lieder,_Op.4/5_Der_Sternenh...",Der Sternenhimmel,5103263,"Zumsteeg,_Emilie",Emilie Zumsteeg,Female,"German composer, music teacher, choir conducto...",2,11,"../../Lieder/scores/Zumsteeg,_Emilie/6_Lieder,...","[voice, piano, piano]",True


In [19]:
scores_df.to_csv('../author-profiling-in-symbolic-music/dataframes/scores_df.csv',index=False)

In [15]:
track_names_mapping_df['track_name_clean'].unique()

array(['piano', 'voice', 'other'], dtype=object)

In [2]:
scores_df=pd.read_csv('../dataframes/scores_df.csv')

In [11]:
pd.set_option('display.max_colwidth', None)    # Do not truncate column content
pd.set_option('display.width', 1000)    


row_4 = scores_df.iloc[4].copy()
row_15 = scores_df.iloc[15].copy()

scores_df.iloc[4] = row_15
scores_df.iloc[15] = row_4

scores_df[['scores_paths' , 'name', 'composer_name',
       'composer_gender', 'desc', 'scores']]


,scores_paths,name,composer_name,composer_gender,desc,scores
0,"../Lieder/scores/Abbott,_Jane_Bingham/_/Just_for_Today/lc6583477.mid",Just for Today,Jane Bingham Abbott,Female,"US pianist, singer and composer",2
1,"../Lieder/scores/Abbott,_Jane_Bingham/_/Think_of_Today/lc6583512.mid",Think of Today,Jane Bingham Abbott,Female,"US pianist, singer and composer",2
2,"../Lieder/scores/Abrams,_Harriett/_/Crazy_Jane/lc6583907.mid",Crazy Jane,Harriett Abrams,Female,"English musician, singer, composer",2
3,"../Lieder/scores/Abrams,_Harriett/_/The_Orphan’s_Prayer/lc6583966.mid",The Orphan’s Prayer,Harriett Abrams,Female,"English musician, singer, composer",2
4,"../Lieder/scores/Barnby,_Joseph/_/To_England/lc6479829.mid",To England,Joseph Barnby,Male,British musician,2
...,...,...,...,...,...,...
1349,"../Lieder/scores/Zumsteeg,_Emilie/6_Lieder,_Op.4/2_Morgenfreude/lc6160298.mid",Morgenfreude,Emilie Zumsteeg,Female,"German composer, music teacher, choir conductor, singer and pianist",11
1350,"../Lieder/scores/Zumsteeg,_Emilie/6_Lieder,_Op.4/3_Religion/lc6162644.mid",Religion,Emilie Zumsteeg,Female,"German composer, music teacher, choir conductor, singer and pianist",11
1351,"../Lieder/scores/Zumsteeg,_Emilie/6_Lieder,_Op.4/4_An_meine_Zither/lc6162666.mid",An meine Zither,Emilie Zumsteeg,Female,"German composer, music teacher, choir conductor, singer and pianist",11
1352,"../Lieder/scores/Zumsteeg,_Emilie/6_Lieder,_Op.4/5_Der_Sternenhimmel/lc6162720.mid",Der Sternenhimmel,Emilie Zumsteeg,Female,"German composer, music teacher, choir conductor, singer and pianist",11


In [20]:
def check_instruments(inst_list):
    if ('other' in inst_list):
        return 'accompanying instrument'
    elif ('piano' in inst_list) and ('voice' in inst_list):
        return 'piano and voice'
    elif ('piano' in inst_list) and ('voice' not in inst_list):
        return 'only piano'

In [22]:
scores_df['track_names'].apply(lambda x : check_instruments(x)).value_counts(normalize=True)

track_names
piano and voice            0.935698
accompanying instrument    0.063562
only piano                 0.000739
Name: proportion, dtype: float64